In [1]:
# import required libraries
import pandas as pd    # Read CSV data from Wigle Wifi Capture
import folium          # Mapping Library

In [2]:
# import CSV file
df = pd.read_csv('./dataset/sample.csv', delimiter = ',', encoding='latin-1', header=1)

In [3]:
# Verify that files have been successfully imported
df.sample(5)

,MAC,SSID,AuthMode,FirstSeen,Channel,RSSI,CurrentLatitude,CurrentLongitude,AltitudeMeters,AccuracyMeters,Type
72,00:1a:4f:18:72:f1,WLAN-001A4F1872F1,[WPA-PSK-TKIP][WPA2-PSK-CCMP],2011-09-16 17:24:28,11,-85,49.422179,11.067460,366,9.487171,WIFI
75,00:15:0c:f3:d9:5a,OBA-Nuernberg,[WPA-PSK-TKIP][WPA2-PSK-CCMP],2011-09-16 17:24:28,11,-95,49.422179,11.067460,366,9.487171,WIFI
32,00:24:01:28:90:65,erdogan,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS],2011-09-16 17:23:59,1,-93,49.425656,11.065185,362,4.641000,WIFI
3,00:a0:f9:18:86:33,0816,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP],2011-09-16 17:23:00,6,-100,49.427670,11.064938,0,1071.000000,WIFI
13,00:08:54:af:02:c6,Mamani,[WPA-PSK-TKIP],2011-09-16 17:23:29,1,-96,49.426128,11.065249,366,4.641000,WIFI


In [4]:
# Create A List Of Valid Parameters
valid = []
for rows in df[['MAC', 'SSID', 'AuthMode', 'FirstSeen', 'Channel', 'RSSI', 'CurrentLatitude', 'CurrentLongitude', 'AltitudeMeters', 'AccuracyMeters', 'Type']].values:
    if (rows[10] == 'WIFI'):
        valid.append(rows)                

In [5]:
# Clean Set by dropping all NA values
validframes = pd.DataFrame(valid).dropna()
validframes.head()

,0,1,2,3,4,5,6,7,8,9,10
0,88:25:2c:32:f3:ae,WLAN-32F349,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS],2011-09-16 17:23:00,6,-87,49.42767,11.064938,0,1071.0,WIFI
1,00:08:54:af:02:c6,Mamani,[WPA-PSK-TKIP],2011-09-16 17:23:00,1,-96,49.42767,11.064938,0,1071.0,WIFI
2,00:23:08:a8:91:8d,EasyBox-A89131,[WPA-PSK-TKIP][WPS],2011-09-16 17:23:00,11,-99,49.42767,11.064938,0,1071.0,WIFI
3,00:a0:f9:18:86:33,0816,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP],2011-09-16 17:23:00,6,-100,49.42767,11.064938,0,1071.0,WIFI
4,14:d6:4d:ff:79:94,dlink,[WPS][WEP],2011-09-16 17:23:00,3,-96,49.42767,11.064938,0,1071.0,WIFI


In [6]:
# Label Columns
validframes.columns = ['MAC', 'SSID', 'AuthMode', 'FirstSeen', 'Channel', 'RSSI', 'CurrentLatitude', 'CurrentLongitude', 'AltitudeMeters', 'AccuracyMeters', 'Type']

In [11]:
# Compute Average of all the latitudes and longitudes in our dataset to find center and set zoom
# You can also hardcode center adddress like
# mymap = folium.Map( location=[34.0522, -118.243683], zoom_start=12)
mymap = folium.Map(location=[ validframes.CurrentLatitude.mean(), validframes.CurrentLongitude.mean() ], 
                   zoom_start=17,
                   tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                   attr = 'Esri',
                   name = 'Esri Satellite',
                   overlay = False,
                   control = True)

In [12]:
# Filter Out Wifi Data
for coord in validframes[['CurrentLatitude','CurrentLongitude', 'SSID', 'Type', 'MAC']].values:
    if ("?" not in str(coord[0])) and ("?" not in str(coord[1])):
        # Set location using Lat-Long(cord[0]-cord[1])
        # Set radius and color of marker 
        # Set data to show on popup
        folium.Marker(location=[coord[0],coord[1]],tooltip=["SSID:", coord[2]], popup=["SSID:", coord[2], "BSSID:", coord[4]],icon=folium.Icon(color='red',prefix='fa',icon='wifi')).add_to(mymap)

In [13]:
# Save MapData To HTML File:
mymap.save('mapdata.html')

In [14]:
%%HTML
<iframe width="100%" height="650" src="mapdata.html"></iframe>